In [1]:
"""
Libraries

"""

import csv
from datetime import datetime
from typing import Callable

import logging

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from scipy.optimize import linear_sum_assignment
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
"""
Setting generic hyperparameters

"""

num_epochs: int = 5#20
batch_size: int = 256    # Should be set to a power of 2.
# Learning rate
lr:         float = 1e-4 # Learning rate used in the IIC paper: lr=1e-4.

"""
GPU utilization

"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Specifications
if torch.cuda.is_available():
    print(f"Number of available devices: {torch.cuda.device_count()}\n",
          f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}\n",
          f"Total GPU memory device 0: {torch.cuda.get_device_properties(0).total_memory/(1024**3):.2f} GB\n",
          f"Total GPU memory device 1: {torch.cuda.get_device_properties(1).total_memory/(1024**3):.2f} GB")

Number of available devices: 2
 Device name: NVIDIA GeForce RTX 2080 Ti
 Total GPU memory device 0: 10.76 GB
 Total GPU memory device 1: 10.76 GB


In [ ]:
'''
Store data to .csv file

'''

# open the file for writing
f = open(f'logs/{datetime.now().strftime("%Y-%m-%d-%H-%M")}.csv', 'w')
# create a CSV writer object
writer = csv.writer(f)
# write the header row to the CSV file
writer.writerow(['epoch', 'loss', 'running_acc', 'acc', 'running_nmi', 'nmi'])

In [4]:
"""
Unsupervised Machine Learning Framework

"""

def train(model, data_loader: DataLoader, criterion: Callable, optimizer: torch.optim, num_epochs: int) -> None:
    """
    Trains a given model using the provided training data, optimizer and loss criterion for a given number of epochs.

    Args:
        model: Neural network model to train.
        data_loader: PyTorch data loader containing the training data.
        criterion: Loss criterion used for training the model.
        optimizer: Optimizer used to update the model's parameters.
        num_epochs: Number of epochs to train the model.

    Returns:
        None
    """

    for epoch in range(num_epochs):

        running_loss = 0.0
        running_acc  = 0.0
        running_nmi  = 0.0

        # Initialize tensors for storing true and predicted labels
        labels_true = torch.zeros(len(data_loader.dataset))
        labels_pred = torch.zeros(len(data_loader.dataset))

        # Loop over the mini-batches in the data loader
        for i, data in enumerate(data_loader):
        
            # Get the inputs and labels for the mini-batch
            inputs, labels = data

            # Use GPU if available
            inputs = inputs.to(device)

            # Image augmentation
            if data_loader.dataset.augment_data:
                inputs_trans = data_loader.dataset.transform_list(inputs)
                # # Flatten input data for the feed forward model
                # inputs       = [inputs.view(inputs.size(0), -1), inputs_trans.view(inputs_trans.size(0), -1)]
                inputs       = [inputs, inputs_trans]
            # else:
                # inputs = inputs.view(inputs.size(0), -1)
        
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass through the model
            if data_loader.dataset.augment_data:
                outputs = [F.softmax(model(inputs[0]), dim=1), F.softmax(model(inputs[1]), dim=1)]
            else:
                outputs = F.softmax(model(inputs), dim=1)

            # Set arguments for objective function
            # kwargs = {key: value for key, value in locals().items() if key in criterion.__code__.co_varnames}
            kwargs = {"model": model, "inputs": inputs, "outputs": outputs}
            kwargs = {key: value for key, value in kwargs.items() if key in criterion.__code__.co_varnames}
            
            # Compute the loss
            loss = criterion(**kwargs)
            # Backward pass through the model and compute gradients
            loss.backward()
        
            # Update the weights
            optimizer.step()

            # Accumulate the loss for the mini-batch
            running_loss += loss.item()

            outputs = outputs[0] if data_loader.dataset.augment_data else outputs

            running_acc  += unsupervised_clustering_accuracy(labels, torch.argmax(outputs.cpu(), dim=1))
            running_nmi  += normalized_mutual_info_score(labels, torch.argmax(outputs.cpu(), dim=1))

            # Store predicted and true labels in tensors
            labels_true[i*len(labels):(i+1)*len(labels)] = labels
            labels_pred[i*len(labels):(i+1)*len(labels)] = torch.argmax(outputs, dim=1)

        acc = unsupervised_clustering_accuracy(labels_true, labels_pred)
        nmi = normalized_mutual_info_score(labels_true, labels_pred)

        # Compute the average loss and accuracy for the epoch and print
        print(f"Epoch {epoch+1} loss: {running_loss/len(data_loader):.4f},\
              running_acc: {running_acc/len(data_loader):.4f}, acc: {acc:.4f},\
              running_nmi: {running_nmi/len(data_loader):.4f}, nmi: {nmi:.4f}")
        # Store data to file
        writer.writerow([epoch+1, running_loss/len(data_loader), running_acc/len(data_loader), acc, running_nmi/len(data_loader), nmi])

def unsupervised_clustering_accuracy(y_true: torch.Tensor, y_pred: torch.Tensor, C: int=121) -> float:
    """
    Computes the unsupervised clustering accuracy between two clusterings.
    Uses the Hungarian algorithm to find the best matching between true and predicted labels.

    Args:
        y_true: true cluster labels as a 1D torch.Tensor
        y_pred: predicted cluster labels as a 1D torch.Tensor
        C:      number of classes

    Returns:
        accuracy: unsupervised clustering accuracy as a float
    """
    # Create confusion matrix
    cm = confusion_matrix(y_pred, y_true, labels=list(range(C)))

    # Compute best matching between true and predicted labels using the Hungarian algorithm
    _, col_ind = linear_sum_assignment(-cm)

    # Reassign labels for the predicted clusters
    y_pred_reassigned = torch.tensor(col_ind)[y_pred.long()]

    # Compute accuracy as the percentage of correctly classified samples
    acc = accuracy_score(y_true, y_pred_reassigned)

    return acc


def test_classifier(model, data_loader: DataLoader) -> None:
    """
    Testing a classifier given the model and a test set.

    Args:
        model: Neural network model to train.
        data_loader: PyTorch data loader containing the test data.
    
    Returns:
        None
    """
    
    # Disable gradient computation, not needed for inference
    model.eval()
    # Initialize tensors for storing true and predicted labels
    y_true = torch.zeros(len(data_loader.dataset))
    y_pred = torch.zeros(len(data_loader.dataset))

    with torch.no_grad():
        # Iterate over the mini-batches in the data loader
        for i, data in enumerate(data_loader):
            # Get the inputs and true labels for the mini-batch and reshape
            inputs, labels_true = data
            
            # # TODO flattening should be done in the feed forward model, else statement should be removed
            # inputs = inputs.view(inputs.size(0), -1)
            
            # Forward pass through the model to get predicted labels
            labels_pred = F.softmax(model(inputs), dim=1)

            # Store predicted and true labels in tensors
            y_pred[i*len(labels_true):(i+1)*len(labels_true)] = torch.argmax(labels_pred, dim=1)
            y_true[i*len(labels_true):(i+1)*len(labels_true)] = labels_true

    # Compute unsupervised clustering accuracy score
    acc = unsupervised_clustering_accuracy(y_true, y_pred)

    print(f"\nThe unsupervised clustering accuracy score of the classifier is: {acc}")

In [5]:
"""

"""

from archt import get_model

# Information Maximizing Self-Augmented Training
from IMSAT import regularized_information_maximization

# Invariant Information Clustering
from IIC import invariant_information_clustering

from datasets.dataset_classes import NDSBDataset, MNISTDataset

In [6]:
"""

"""

# Create the train and test datasets
train_dataset = NDSBDataset(train=True, augment_data=True)
test_dataset  = NDSBDataset(train=False)

# Create the train and test data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
# Initialize model
model = get_model("cnn").to(device)

# Initialize loss function, and optimizer
criterion = invariant_information_clustering
optimizer = optim.Adam(model.parameters(), lr=lr)

# Store metadata to .log file
logger = logging.getLogger(__name__)
# Set the logging level
logger.setLevel(logging.INFO)
# Add handler to the logger
logger.addHandler(logging.FileHandler(f'logs/{datetime.now().strftime("%Y-%m-%d-%H-%M")}.log'))

# Write metadata to .log file
logger.info(f'Optimization criterion: {criterion.__name__}')
logger.info(f'Learning rate: {lr}')
logger.info(f'Number of epochs: {num_epochs}')
logger.info(f'Batch size: {batch_size}')
logger.info(f'Optimizer: {optimizer}')
logger.info(f'Model: {model}')

# Train the model
train(model, train_loader, criterion, optimizer, num_epochs)

# Close data file
f.close()

# Test model
model.cpu()

test_classifier(model, test_loader)

Model specifications: CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_bn1): BatchNorm2d(16, eps=2e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn3): BatchNorm2d(16, eps=2e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=183184, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=121, bias=True)
  (fc_bn1): BatchNorm1d(1024, eps=2e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)
Epoch 1 loss: -0.9236,              running_acc: 0.1985, acc: 0.1574,              running_nmi: 0.4267, nmi: 0.2435
Epoch 2 loss: -1.1152,              running_acc: 0.2153, acc: 0.1734,              running_nmi: 0.4608, nmi: 0.2667
Epoch 3 loss: -1.2357,        